In [1]:
import numpy
import pandas as pd
import scipy
import PIL
import glob, os
import time
from keras.models import Model, load_model
from keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras import backend as K
from keras.optimizers import Adam



DESIRED_WIDTH = 128
DESIRED_HEIGHT = 128

c:\users\kevin\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [1]:

flag = 0

count=-1
labelsLocation = "C://Users//Kevin//Workspaces//Data//dogIdentification//labels.csv"
fileImagesLocation = "C://Users//Kevin//Workspaces//Data//train"




start = time.time()
os.chdir(fileImagesLocation)

train_y = []    
labels=pd.read_csv(labelsLocation, sep=',')

#Building train_x
for file in glob.glob("*.jpg"):
    count += 1
    
    original_image = PIL.Image.open(file)
    transformed_image = original_image.resize( (DESIRED_HEIGHT, DESIRED_WIDTH) )
    tempArray = numpy.array(transformed_image)
    tempArray = numpy.expand_dims(tempArray, axis = 0)

    if flag == 0:
        train_x = tempArray
        flag = 1
    else:
        train_x = numpy.append(train_x, tempArray, axis=0)
        

    #building train_y part 1
    for x in range(labels.shape[0]):
        tempFlag = 0
        if labels.iloc[x,0] == file[:-4]:
            train_y.append(labels.iloc[x,1])
            labels.drop([x])
            tempFlag = 1
            break
    if tempFlag == 0:
        train_y.append("unknown")
    if count % 100 == 0:
        print(str(count) + " records have been processed")
        
count = 0

for i in train_y:
    if i == "unknown":
        count +=1

uniqueLabels = []
for i in train_y:
    if i not in uniqueLabels:
        uniqueLabels.append(i)
        
count = 0
uniqueLabelMaps = []

uniqueLabels.sort()
for i in uniqueLabels:
    uniqueLabelMaps.append([i, count])
    count += 1
    

for i in range(len(train_y)):
    for j in uniqueLabelMaps:
        if train_y[i] == j[0]:
            train_y[i] = j[1]
            break
            
train_y = to_categorical(train_y)            
            
train_x = train_x /255
os.chdir("C://Users//Kevin//Workspaces//Data//dogIdentification")
numpy.save("train_x", train_x)
numpy.save("train_y", train_y)
end = time.time()
print(end - start)



c:\users\kevin\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


0 records have been processed
100 records have been processed
200 records have been processed
300 records have been processed
400 records have been processed
500 records have been processed
600 records have been processed
700 records have been processed
800 records have been processed
900 records have been processed
1000 records have been processed
1100 records have been processed
1200 records have been processed
1300 records have been processed
1400 records have been processed
1500 records have been processed
1600 records have been processed
1700 records have been processed
1800 records have been processed
1900 records have been processed
2000 records have been processed
2100 records have been processed
2200 records have been processed
2300 records have been processed
2400 records have been processed
2500 records have been processed
2600 records have been processed
2700 records have been processed
2800 records have been processed
2900 records have been processed
3000 records have been

In [53]:
testLocation = "C://Users//Kevin//Workspaces//Data//test"
os.chdir(testLocation)

DESIRED_WIDTH = 128
DESIRED_HEIGHT = 128

start = time.time()

flag = 0
count = 0
test_y_column1 = []
#Building test_x
for file in glob.glob("*.jpg"):
    count += 1
    
    test_y_column1.append(file[:-4])
    
    original_image = PIL.Image.open(file)
    transformed_image = original_image.resize( (DESIRED_HEIGHT, DESIRED_WIDTH) )
    tempArray = numpy.array(transformed_image)
    tempArray = numpy.expand_dims(tempArray, axis = 0)

    if flag == 0:
        test_x = tempArray
        flag = 1
    else:
        test_x = numpy.append(test_x, tempArray, axis=0)
    if count % 100 == 0:
        print(str(count) + " records have been processed")

test_x = test_x /255
end = time.time()
print(end - start)

numpy.save("test_x", test_x)
numpy.save("test_y_column1", test_y_column1)

100 records have been processed
200 records have been processed
300 records have been processed
400 records have been processed
500 records have been processed
600 records have been processed
700 records have been processed
800 records have been processed
900 records have been processed
1000 records have been processed
1100 records have been processed
1200 records have been processed
1300 records have been processed
1400 records have been processed
1500 records have been processed
1600 records have been processed
1700 records have been processed
1800 records have been processed
1900 records have been processed
2000 records have been processed
2100 records have been processed
2200 records have been processed
2300 records have been processed
2400 records have been processed
2500 records have been processed
2600 records have been processed
2700 records have been processed
2800 records have been processed
2900 records have been processed
3000 records have been processed
3100 records have b

In [2]:
labelsLocation = "C://Users//Kevin//Workspaces//Data//dogIdentification//labels.csv"
labels=pd.read_csv(labelsLocation, sep=',')
uniqueLabels = []
for x in range(labels.shape[0]):
    if labels.iloc[x,1] not in uniqueLabels:
        uniqueLabels.append(labels.iloc[x,1])
uniqueLabels.sort()

def inception2B(x, nFilter = 32):
    conv1 = Conv2D(nFilter, (1, 1), padding='same', activation='relu')(x)
      
    conv2 = Conv2D(nFilter, (1, 1), padding='same', activation='relu')(x)
    conv21 = Conv2D(nFilter, (3, 3), padding='same', activation='relu')(conv2)
      
    conv3 = Conv2D(nFilter, (1, 1), padding='same', activation='relu')(x)
    conv31 = Conv2D(nFilter, (5, 5), padding='same', activation='relu')(conv3)
     
    conv4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    conv41 = Conv2D(nFilter, (1,1), padding='same', activation='relu')(conv4)
  
    c2 = Concatenate(axis=-1)([conv1, conv21, conv31, conv41])
    return c2

def inception2BN(x, n = 32):
    conv1 = Conv2D(n, (1, 1), padding='same', activation='relu')(x)
    conv1batchNorm=BatchNormalization()(conv1)
    
    conv2 = Conv2D(n, (1, 1), padding='same', activation='relu')(x)
    conv2batchNorm=BatchNormalization()(conv2)
  
    conv21 = Conv2D(n, (3, 3), padding='same', activation='relu')(conv2batchNorm)
      
    conv3 = Conv2D(n, (1, 1), padding='same', activation='relu')(x)
    conv3batchNorm=BatchNormalization()(conv3)
    conv31 = Conv2D(n, (5, 5), padding='same', activation='relu')(conv3batchNorm)
     
    conv4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    conv41 = Conv2D(n, (1,1), padding='same', activation='relu')(conv4)
  
    c = Concatenate(axis=1)([conv1batchNorm, conv21, conv31, conv41])
    return c

def model2():
    input=Input(shape=(DESIRED_HEIGHT, DESIRED_WIDTH, 3))
    dropoutInput=Dropout(.4)(input)
    inceptionFirst=inception2BN(dropoutInput)
    maxPool1=MaxPooling2D((2,2),strides=(2,2),padding='same')(inceptionFirst)
    inceptionSecond=inception2BN(maxPool1)
    maxPool2=MaxPooling2D((2,2),strides=(2,2),padding='same')(inceptionSecond)    
    flattenedLayer = Flatten()(maxPool2)
    dropout = Dropout(.4)(flattenedLayer)
    temp3=Dense(128,activation='relu')(dropout) 
    temp3BN = BatchNormalization()(temp3)
    output_m1=Dense(len(uniqueLabels),activation='softmax')(temp3BN) 
    model=Model(input,output_m1) 
    
    return model


In [30]:
os.chdir("C://Users//Kevin//Workspaces//Data//dogIdentification")
train_x = numpy.load("train_x.npy")
train_x = train_x.reshape(train_x.shape[0], DESIRED_HEIGHT, DESIRED_WIDTH, 3).astype( 'float32' )

train_y = numpy.load("train_y.npy")



K.clear_session()
model=model2()
highLearningAdam = Adam(lr=0.001)
model.compile(optimizer=highLearningAdam,loss='categorical_crossentropy',metrics=['accuracy'])
checkpoint = ModelCheckpoint('dogbreedIdentification.h5', monitor='val_loss', save_best_only=True)
cb = [checkpoint]
model.summary()  
model.fit(train_x, train_y, validation_split=.2, callbacks=cb epochs=20, batch_size=32)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 128, 128, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 128, 32) 128         dropout_1[0][0]                  
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 128, 128, 32) 128         dropout_1[0][0]                  
__________________________________________________________________________________________________
batch_norm

ResourceExhaustedError: OOM when allocating tensor with shape[5,32,512,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: concatenate_1/concat = ConcatV2[N=4, T=DT_FLOAT, Tidx=DT_INT32, _class=["loc:@training/Adam/gradients/concatenate_1/concat_grad/Slice_2"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_1/Relu, conv2d_3/Relu, conv2d_5/Relu, conv2d_6/Relu, concatenate_1/concat-4-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: dense_2/BiasAdd/_547 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_973_dense_2/BiasAdd", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated

In [3]:
os.chdir("C://Users//Kevin//Workspaces//Data//test")
test_x = numpy.load("test_x.npy")
ycolumn1 = numpy.load("test_y_column1.npy")

K.clear_session()
model=model2()
highLearningAdam = Adam(lr=0.001)
model.compile(optimizer=highLearningAdam,loss='categorical_crossentropy',metrics=['accuracy'])
model.load_weights('dogbreedIdentification.h5')

In [4]:
finalOutput = []

for i in range(test_x.shape[0]):
    predictions = model.predict(test_x[i:i+1])
    temp = [ycolumn1[i]]
    for prediction in predictions:
        for number in prediction:
            temp.append(number)
    finalOutput.append(temp)

tempList = ["id"]
for name in uniqueLabels:
    tempList.append(name)


submissionListOfLists = [tempList]
for row in finalOutput:
    submissionListOfLists.append(row)
finalOutput = pd.DataFrame(numpy.array(submissionListOfLists))
finalOutput.to_csv("submission.csv", header = False, index=False)

In [ ]:
'''
The last bit of codes creates the submission file in the proper format with the proper row ids and column names, yielding a
prediction column space for each id.

Looking over the poor performance of the model generalizing, and looking through the data, I can think of a few things that
are absolutely critical to making the model more effective. These include the following:

1) Using a bigger image size, preferably 256x256
    - this is limited by computing power and the resource. Using a 1.4 gb gpu was not sufficient for this project, requiring
        a migration to colab
    - this is expected to capture more defining curves of the dogs
2) Background blackout, due to confounding image colours
    - this is beyond my personal scope of ability. It first requires classifying the image in the background as the target
        using image recognition, then zeroing out all other pixels
    - I expect this to be the most important thing for this image, as in identifying dogs we need to use the full RGB spectrum,
        and looking at the images, our feature space for 123 target variables is severely hampered by the diversity of color 
        backgrounds.
3) More data
    - an obvious solution is image augmentation, but in this case, with 123 target variables 10,000 samples might not be truly
        enough, especially given the diversity of environments the photos were taken in. This can be circumvented using 
        solution #2, as I believe that to be the most important thing
4) Better model.
    - Without 1, 2, or 3, this will not alone give the best performance, but finer tuning can be performed once a cross validation
        goes above .1, which I managed to get on a single test.


'''